In [2]:
import pandas as pd
import numpy as np


In [37]:
baseline_period = (1960,1990)
future_period = (2050,2060)

values = np.linspace(0,1799,150*12,dtype=int)

all_months = pd.date_range("01 January 1951",end="01 January 2101",freq='M')
climate_data = pd.Series(values,index=all_months)


def pandas_slice_year(data, start_year, end_year):
    return data[(data.index.year >= start_year) & (data.index.year <= end_year)]

historical_data = pandas_slice_year(climate_data, baseline_period[0], baseline_period[1])

In [52]:
s = pd.Series(np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13]))
s.index = pd.date_range("01 January 2023",periods=14,freq='D')
pd.DataFrame(s.rename("drybulb_C")).resample('W').sum().index.month


Index([1, 1, 1], dtype='int32')

In [74]:

single = pd.Series(np.ones(8760))
single.index = pd.date_range("01 January 2023",periods=8760,freq="h")


months = list(range(1, 12 + 1, 1))

month_hours = dict(zip(months, single.resample('M').sum().tolist()))  # hours

for k,v in month_hours.items():
    print(k)

1
2
3
4
5
6
7
8
9
10
11
12


In [75]:
pd.Series(range(0,8760))

0          0
1          1
2          2
3          3
4          4
        ... 
8755    8755
8756    8756
8757    8757
8758    8758
8759    8759
Length: 8760, dtype: int64

In [82]:

def read_epw(path_data):
    tmy_labels = [
        'year', 'month', 'day', 'hour', 'minute', 'datasource', 'drybulb_C',
        'dewpoint_C', 'relhum_percent', 'atmos_Pa', 'exthorrad_Whm2',
        'extdirrad_Whm2', 'horirsky_Whm2', 'glohorrad_Whm2', 'dirnorrad_Whm2',
        'difhorrad_Whm2', 'glohorillum_lux', 'dirnorillum_lux',
        'difhorillum_lux', 'zenlum_lux', 'winddir_deg', 'windspd_ms',
        'totskycvr_tenths', 'opaqskycvr_tenths', 'visibility_km',
        'ceiling_hgt_m', 'presweathobs', 'presweathcodes', 'precip_wtr_mm',
        'aerosol_opt_thousandths', 'snowdepth_cm', 'days_last_snow', 'Albedo',
        'liq_precip_depth_mm', 'liq_precip_rate_Hour'
    ]

    df = pd.read_csv(path_data,
                     skiprows=8,
                     header=None,
                     index_col=False,
                     usecols=list(range(0, 35)),
                     names=tmy_labels)#.drop('datasource', axis=1)

    df['hour'] = df['hour'].astype(int)
    if df['hour'][0] == 1:
        print('TMY file hours reduced from 1-24h to 0-23h')
        df['hour'] = df['hour'] - 1
    else:
        print('TMY file hours maintained at 0-23hr')
    df['minute'] = 0
    return df


In [113]:
from skyfield import api, almanac
import datetime

de_file = "/Users/jmccarty/GitHub/pyepwmorph/morpher/assets/de421.bsp"
latitude = 49.078646574052506
longitude = -122.63912546915324

df = read_epw("/Users/jmccarty/Downloads/CHE_GE_Geneva/CHE_GE_Geneva.Intl.AP.067000_TMYx.2004-2018.epw")
df.set_index(pd.date_range("01-01-2023",freq="h",periods=8760), inplace=True, drop=True)
ts = api.load.timescale()
eph = api.load(de_file)

location = api.Topos(latitude, longitude)

year = 2023

t0 = ts.utc(year - 1, 12, 31, 0)
t1 = ts.utc(year + 1, 1, 2, 0)

t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(eph, location))
times = pd.Series(t.utc_datetime()).rename('datetimes')
times = times + datetime.timedelta(hours=-8)
keys = pd.Series(y).rename('Rise_Set')
keys = pd.Series(np.where(keys == 0, 'Sunset', 'Sunrise')).rename('Rise_Set')
join = pd.concat([times, keys], axis=1)
join.set_index(join['datetimes'], inplace=True)
join['year'] = join['datetimes'].dt.year
join['month'] = join['datetimes'].dt.month
join['day'] = join['datetimes'].dt.day
join['hour'] = join['datetimes'].dt.hour
join['minute'] = 0
join = join[join['year'] == year]
join['Timestamp'] = join.apply(lambda row: datetime.datetime(row.year, row.month, row.day, row.hour), axis=1)
join.set_index('Timestamp', inplace=True)
join_sub = pd.DataFrame(join['Rise_Set'])
join_sub['dtime'] = join.index
df['dtime'] = df.index
df = df.merge(join_sub, how='left', left_on='dtime', right_on='dtime')
df['Rise_Set'] = df['Rise_Set'].fillna('Neither')
df.set_index(df['dtime'], inplace=True)

pd.Series(df['Rise_Set'].values).head(12)

# .loc[0]==0
# .loc[8759]==1
# Sunrise=2
# Sunset=3
# Neither=4

TMY file hours reduced from 1-24h to 0-23h


0     Neither
1     Neither
2     Neither
3     Neither
4     Neither
5     Neither
6     Neither
7     Neither
8     Sunrise
9     Neither
10    Neither
11    Neither
dtype: object

In [92]:
pd.Series(t.utc_datetime()).rename('datetimes')

0     2022-12-31 07:13:22.043407+00:00
1     2022-12-31 15:44:46.919578+00:00
2     2023-01-01 07:13:24.268219+00:00
3     2023-01-01 15:45:42.482986+00:00
4     2023-01-02 07:13:23.623202+00:00
                    ...               
729   2023-12-30 15:43:41.266504+00:00
730   2023-12-31 07:13:21.151077+00:00
731   2023-12-31 15:44:33.953766+00:00
732   2024-01-01 07:13:24.074899+00:00
733   2024-01-01 15:45:28.945180+00:00
Name: datetimes, Length: 734, dtype: datetime64[ns, UTC]

In [145]:
import pvlib
from timezonefinder import TimezoneFinder

latitude = 49.078646574052506
longitude = -122.63912546915324
tz = TimezoneFinder().timezone_at(lng=longitude, lat=latitude)

def build_sunrise_sunset(location):
    latitude = location['latitude']
    longitude = location['longitude']
    tz = TimezoneFinder().timezone_at(lng=longitude, lat=latitude)
    times =  pd.date_range('2019-01-01 00:00:00', periods=8760, freq='H', tz=tz)
    rise_set = pvlib.location.Location(latitude, longitude, tz=tz).get_sun_rise_set_transit(times, method='spa')
    sunrise = pd.Series(rise_set.index.hour == pd.to_datetime(rise_set['sunrise']).apply(lambda x: x.hour))
    sunset = pd.Series(rise_set.index.hour == pd.to_datetime(rise_set['sunset']).apply(lambda x: x.hour))

    # .loc[0]==1
    # .loc[8759]==4
    # Sunrise=2
    # Sunset=3
    # Neither=0
    sunrise = np.where(sunrise==True,2,0)
    sunset = np.where(sunset==True,3,0)
    sunrise_sunset = sunset + sunset
    sunrise_sunset[0] = 1
    sunrise_sunset[-1] = 4
    sunrise_sunset

array([1, 0, 0, ..., 0, 0, 4])

In [149]:
all_model_vars = ['tas','tas', 'tasmax', 'tasmin', 'clt', 'psl', 'pr', 'huss', 'vas', 'uas', 'rsds']

list(set(all_model_vars))

['psl', 'clt', 'tasmax', 'uas', 'tasmin', 'tas', 'rsds', 'vas', 'pr', 'huss']

In [180]:
def read_epw_string(filepath):
    with open(filepath,"r") as fp:
        file_content = fp.readlines()
    return file_content

def epw_find_header_length(file_content):
    csvreader = csv.reader(file_content, delimiter=',', quotechar='"')
    for i,row in enumerate(csvreader):
        if row[0].isdigit():
            break
    return i
    
def read_epw_header(file_content):
    d={}
    csvreader = csv.reader(file_content, delimiter=',', quotechar='"')
    for n, row in enumerate(csvreader):
        if n==epw_find_header_length(file_content):
            break
        else:
            d[row[0]]=row[1:]

    return d

def epw_location(file_content):
    location_line = file_content[0].replace("\n","").split(",")
    location_keys = ['title','site','province','country_code','usaf','usaf','longitude','latitude','utc_offset','elevation']
    location_dict = dict(zip(location_keys,location_line))
    location_dict['longitude'] = float(location_dict['longitude'])
    location_dict['latitude'] = float(location_dict['latitude'])
    location_dict['elevation'] = float(location_dict['elevation'])
    location_dict['utc_offset'] = float(location_dict['utc_offset'])
    return location_dict

In [204]:
import re
epw_content = read_epw_string("/Users/jmccarty/Downloads/CHE_GE_Geneva/CHE_GE_Geneva.Intl.AP.067000_TMYx.2004-2018.epw")
read_epw_header(epw_content)
subl = [l for l in epw_content if "Period of Record" in l][0].replace("\n","").split("Period of Record")[-1].replace("="," ").replace(";"," ").replace("-"," ")
years = np.array(list(set([int(y) for y in re.findall(r'\d{4}', subl)])))
baseline_range = (years.min(),years.max())

2004

## Model

In [1]:
%load_ext autoreload
%autoreload 2

In [220]:
import xarray as xr
import xclim
import numpy as np
import pandas as pd
import sys
module_path = "/Users/jmccarty/GitHub/pyepwmorph"
sys.path.insert(0, module_path)
from pyepwmorph.tools import io
import pyepwmorph.models.access as pma
import pyepwmorph.models.coordinate as pmc
import pyepwmorph.models.assemble as pmas
import pyepwmorph.tools.workflow as pw
import pyepwmorph.tools.io as pio

In [45]:
data = np.arange(6).reshape(2, 3)
labels = ["a", "b", "c"]
ds = xr.Dataset({"A": (["x", "y"], data), "y": labels})
ds

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * y        (y) <U1 'a' 'b' 'c'
Dimensions without coordinates: x
Data variables:
    A        (x, y) int64 0 1 2 3 4 5

In [398]:
data_dict = pma.access_cmip6_data(['ACCESS-CM2', 'CanESM5', 'TaiESM1'], 'historical', 'tas')


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [403]:
data_dict.keys()

dict_keys(['CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.Amon.gn', 'CMIP.CCCma.CanESM5.historical.Amon.gn', 'CMIP.AS-RCEC.TaiESM1.historical.Amon.gn'])

In [405]:
data_dict['CMIP.AS-RCEC.TaiESM1.historical.Amon.gn'].time


<xarray.DataArray 'time' (time: 1980)>
array([      0,     708,    1416, ..., 1443192, 1443924, 1444656])
Coordinates:
    height   float64 ...
  * time     (time) int64 0 708 1416 2148 ... 1442460 1443192 1443924 1444656
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar:       noleap
    long_name:      time
    standard_name:  time
    units:          hours since 1850-01-16 12:00:00.000000

In [388]:
data_dict_path = pma.access_cmip6_data(['ACCESS-CM2', 'CanESM5', 'TaiESM1'], 'ssp126', 'tas')


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [399]:
data_dict['ScenarioMIP.AS-RCEC.TaiESM1.ssp126.Amon.gn'].time

KeyError: 'ScenarioMIP.AS-RCEC.TaiESM1.ssp126.Amon.gn'

In [393]:
data_dict['ScenarioMIP.CCCma.CanESM5.ssp126.Amon.gn'].time

<xarray.DataArray 'time' (time: 3432)>
array([      0,     708,    1416, ..., 2503152, 2503884, 2504616])
Coordinates:
    height   float64 ...
  * time     (time) int64 0 708 1416 2148 ... 2502420 2503152 2503884 2504616
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar:       noleap
    long_name:      time
    standard_name:  time
    units:          hours since 2015-01-16 12:00:00.000000

In [390]:
datasets = pmc.coordinate_cmip6_data(49.100, -121.300, 'ssp126', 'tas', data_dict)

ScenarioMIP.CCCma.CanESM5.ssp126.Amon.gn


ValueError: Date range is invalid for pandas DatetimeIndex, try using `use_cftime=True`.

In [200]:
datasets['ScenarioMIP.AS-RCEC.TaiESM1.ssp126.Amon.gn'].to_dataframe()[['tas']]


,tas
time,
2015-01-01,281.268982
2015-02-01,280.338806
2015-03-01,283.186798
2015-04-01,285.167694
2015-05-01,287.643860
...,...
2100-08-01,295.585205
2100-09-01,292.753479
2100-10-01,289.429749


In [211]:
ds_ens = pmas.build_cmip6_ensemble([1,50,99], 'tas', datasets)

# data_var = ds_ens[1].drop('percentiles').values


In [217]:
c = pw.compile_climate_model_data(['ACCESS-CM2', 'CanESM5', 'TaiESM1'], 'historical', 'tas', 49.100, -121.300, [1,50,99])

[autoreload of pyepwmorph.tools.workflow failed: Traceback (most recent call last):
  File "/Users/jmccarty/.pyenv/versions/3.10.10/envs/pyepwmorph/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/jmccarty/.pyenv/versions/3.10.10/envs/pyepwmorph/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/Users/jmccarty/.pyenv/versions/3.10.10/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/jmccarty/GitHub/pyepwmorp


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
CMIP.AS-RCEC.TaiESM1.historical.Amon.gn███| 100.00% [3/3 00:06<00:00]
CMIP.CCCma.CanESM5.historical.Amon.gn
CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.Amon.gn


## morph

In [232]:
data_dict = pw.iterate_compile_model_data(['historical','ssp126'],
                                          ['tas','tasmax','tasmin'],
                                          ['ACCESS-CM2', 'CanESM5', 'TaiESM1'],
                                          49.100, 
                                          -121.300,
                                          [1,50,99]
                                          )



Compiling model data for 'historical' and 'tas'.

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
CMIP.AS-RCEC.TaiESM1.historical.Amon.gn███| 100.00% [3/3 00:06<00:00]
CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.Amon.gn
CMIP.CCCma.CanESM5.historical.Amon.gn

Compiling model data for 'historical' and 'tasmax'.

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.Amon.gn.00% [2/2 00:00<00:00]
CMIP.CCCma.CanESM5.historical.Amon.gn

Compiling model data for 'historical' and 'tasmin'.

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.Amon.gn.00% [2/2 00:00<00:00]
CMIP.CCCma.CanESM5.historical.Amon.gn

Compil

In [268]:
filepath="/Users/jmccarty/Downloads/CHE_GE_Geneva/CHE_GE_Geneva.Intl.AP.067000_TMYx.2004-2018.epw"
epw_df = pio.read_epw_dataframe(filepath)
pw.morph_epw(epw_df, ['Temperature'], (1970,1980), (2040,2050), data_dict, 'ssp126', 50)

TMY file hours reduced from 1-24h to 0-23h


2023-01-01 00:00:00    3.38
2023-01-01 01:00:00    3.16
2023-01-01 02:00:00    2.83
2023-01-01 03:00:00    2.72
2023-01-01 04:00:00    2.28
                       ... 
2023-12-31 19:00:00    7.59
2023-12-31 20:00:00    7.99
2023-12-31 21:00:00    8.11
2023-12-31 22:00:00    8.22
2023-12-31 23:00:00    7.44
Freq: H, Name: drybulb_C, Length: 8760, dtype: float64

In [270]:

morphed_dict = {'relhum_percent':0}


if ('relhum_percent' in morphed_dict.keys()) & ('drybulb_C' in morphed_dict.keys()):
    # do morphing
    print(90)
else:
    print('')

In [248]:
data_dict['ssp126']['tasmax'][50].reanme("")

time
2015-01-01    251.409271
2015-02-01    235.769402
2015-03-01    223.550354
2015-04-01    215.954697
2015-05-01    217.017647
                 ...    
2100-08-01    218.382172
2100-09-01    213.396942
2100-10-01    227.796471
2100-11-01    243.932037
2100-12-01    255.848366
Freq: MS, Name: 50, Length: 1032, dtype: float64

In [281]:
pd.concat([data_dict['historical']['tas'][50],data_dict['ssp126']['tas'][50]])

time
1960-01-01    246.460693
1960-02-01    234.672745
1960-03-01    221.656555
1960-04-01    221.082031
1960-05-01    214.585983
                 ...    
2100-08-01    216.847519
2100-09-01    212.713211
2100-10-01    224.233627
2100-11-01    239.096161
2100-12-01    252.511719
Freq: MS, Name: 50, Length: 1692, dtype: float64

In [363]:
my_epw = pio.Epw(filepath)
my_epw.detect_baseline_range()

(2004, 2018)

In [ ]:
user_variables = ['Temperature','Humidity']
baseline_range = my_epw.detect_baseline_range()
future_range = (2030,2050)

pathway = 'ssp245'
percentile = 50

data_dict = pw.iterate_compile_model_data(['historical',pathway],
                                          ['tas','tasmax','tasmin','huss'],
                                          ['ACCESS-CM2', 'CanESM5', 'TaiESM1'],
                                          49.100, 
                                          -121.300,
                                          [1,50,99]
                                          )

dydcy5-qezbin-zaqciT

# model_data_dict = 
# pw.morph_epw(filepath, user_variables, baseline_range, future_range, model_data_dict, pathway, percentile)

In [383]:
new_epw = pw.morph_epw(filepath, user_variables, baseline_range, future_range, data_dict, pathway, percentile)

In [386]:
my_epw.dataframe.head()

,year,month,day,hour,minute,datasource,drybulb_C,dewpoint_C,relhum_percent,atmos_Pa,...,ceiling_hgt_m,presweathobs,presweathcodes,precip_wtr_mm,aerosol_opt_thousandths,snowdepth_cm,days_last_snow,Albedo,liq_precip_depth_mm,liq_precip_rate_Hour
2023-01-01 00:00:00,2015,1,1,1,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-0.3,-4.2,72,98194,...,77777,9,999999999,6,0.085,0,88,0.220,0.0,0.0
2023-01-01 01:00:00,2015,1,1,2,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-0.7,-4.4,73,98243,...,77777,9,999999999,6,0.085,0,88,0.247,0.0,0.0
2023-01-01 02:00:00,2015,1,1,3,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-1.3,-4.7,75,98298,...,77777,9,999999999,6,0.085,0,88,0.287,0.0,0.0
2023-01-01 03:00:00,2015,1,1,4,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-1.5,-4.4,78,98313,...,77777,9,999999999,6,0.085,0,88,0.300,0.0,0.0
2023-01-01 04:00:00,2015,1,1,5,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-2.3,-4.8,81,98307,...,77777,9,999999999,6,0.085,0,88,0.353,0.0,0.0


In [387]:
new_epw.dataframe.head()

,year,month,day,hour,minute,datasource,drybulb_C,dewpoint_C,relhum_percent,atmos_Pa,...,ceiling_hgt_m,presweathobs,presweathcodes,precip_wtr_mm,aerosol_opt_thousandths,snowdepth_cm,days_last_snow,Albedo,liq_precip_depth_mm,liq_precip_rate_Hour
2023-01-01 00:00:00,2015,1,1,1,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,0.93,-4.2,81.20,98194,...,77777,9,999999999,6,0.085,0,88,0.220,0.0,0.0
2023-01-01 01:00:00,2015,1,1,2,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,0.54,-4.4,82.33,98243,...,77777,9,999999999,6,0.085,0,88,0.247,0.0,0.0
2023-01-01 02:00:00,2015,1,1,3,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-0.06,-4.7,84.58,98298,...,77777,9,999999999,6,0.085,0,88,0.287,0.0,0.0
2023-01-01 03:00:00,2015,1,1,4,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-0.25,-4.4,87.97,98313,...,77777,9,999999999,6,0.085,0,88,0.300,0.0,0.0
2023-01-01 04:00:00,2015,1,1,5,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9,-1.05,-4.8,91.35,98307,...,77777,9,999999999,6,0.085,0,88,0.353,0.0,0.0
